In [13]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

import warnings
warnings.filterwarnings("ignore")

In [14]:
master = pd.read_csv('master_data_all_factors.csv')
table_3_summary = pd.read_excel('./result/table3.xlsx')

In [15]:
top_3_actual = table_3_summary.sort_values('Actual', ascending=False).head(3)
top_3_actual

,Model,Actual,FS Average,FS Median,IS Average,IS Median,OS Average,OS Median
1,"Mkt, SMB, HML_s, RMW_s, CMA_s, UMD_s",0.198995,0.214116,0.211034,0.230197,0.224304,0.190063,0.184377
3,"Mkt, S-F, H_s-F, R_s-F, C_s-F, U_s-F",0.148570,0.162039,0.159611,0.175520,0.170653,0.140439,0.135646
0,"Mkt, SMB, HML, RMW, CMA, UMD",0.141268,0.155580,0.153001,0.170338,0.165171,0.133163,0.128445


In [16]:
top_3_actual[top_3_actual['Model'] == 'Mkt, SMB, HML, RMW, CMA, UMD']['Actual'][0]

0.1412679393510698

In [17]:
col_3_factors = [model.split(', ') for model in top_3_actual['Model'].values]
col_3_factors

[['Mkt', 'SMB', 'HML_s', 'RMW_s', 'CMA_s', 'UMD_s'],
 ['Mkt', 'S-F', 'H_s-F', 'R_s-F', 'C_s-F', 'U_s-F'],
 ['Mkt', 'SMB', 'HML', 'RMW', 'CMA', 'UMD']]

### Table 4

In [5]:
# Generate synthetic data for demonstration
np.random.seed(42)

def squared_sharpe_ratio(sample, factor, weights=None):
    if weights is None:
        mu_f = sample[factor].mean().values.reshape(-1, 1)
    else:
        mu_f = (sample[factor] * weights).sum(axis=1)

    sigma_f = sample[factor].cov().values
    sigma_f_inv = np.linalg.inv(sigma_f)

    # maximum squared Sharpe ratio
    sh2_f = mu_f.T @ sigma_f_inv @ mu_f

    return sh2_f[0, 0]

def simulate_sharpe_ratios(returns, row_factor, col_3_factors, n_simulations=100):
    n_months = len(returns)
    n_pairs = n_months // 2
    pairs = [(i, i+1) for i in range(0, n_months, 2)]

    fs_sharpe_ratios_diff = [[], [], []]
    is_sharpe_ratios_diff = [[], [], []]
    os_sharpe_ratios_diff = [[], [], []]

    for _ in range(n_simulations):
        # Full-Sample (FS) simulation for row model
        fs_sample_row = returns.sample(n=n_months, replace=True)
        fs_sharpe_ratio_row = squared_sharpe_ratio(fs_sample_row, row_factor)

        for i, col_factor in enumerate(col_3_factors):
            if row_factor == col_factor:
                continue
            # Full-Sample (FS) simulation for column model
            fs_sample_col = returns.sample(n=n_months, replace=True)
            fs_sharpe_ratio_col = squared_sharpe_ratio(fs_sample_col, col_factor)

            # Store the difference FS
            fs_sharpe_ratios_diff[i].append(fs_sharpe_ratio_col - fs_sharpe_ratio_row)

            # In-Sample (IS) and Out-of-Sample (OS) simulations
            sampled_pairs = np.random.choice(range(n_pairs), size=n_pairs, replace=True)
            is_sample_indices = []
            os_sample_indices = []

            for pair_index in sampled_pairs:
                month1, month2 = pairs[pair_index]
                if np.random.rand() < 0.5:
                    is_sample_indices.append(month1)
                    os_sample_indices.append(month2)
                else:
                    is_sample_indices.append(month2)
                    os_sample_indices.append(month1)

            is_sample_row = returns.iloc[is_sample_indices]
            os_sample_row = returns.iloc[os_sample_indices]

            is_sample_col = returns.iloc[is_sample_indices]
            os_sample_col = returns.iloc[os_sample_indices]

            # Calculate IS Sharpe ratio for row and column models
            is_sharpe_ratio_row = squared_sharpe_ratio(is_sample_row, row_factor)
            is_sharpe_ratio_col = squared_sharpe_ratio(is_sample_col, col_factor)
            is_sharpe_ratios_diff[i].append(is_sharpe_ratio_col - is_sharpe_ratio_row)

            # Calculate OS Sharpe ratio using IS weights for row model
            is_weights_row = tangency_portfolio_weights(is_sample_row, row_factor)
            os_sharpe_ratio_row = squared_sharpe_ratio(os_sample_row, row_factor, is_weights_row)

            # Calculate OS Sharpe ratio using IS weights for column model
            is_weights_col = tangency_portfolio_weights(is_sample_col, col_factor)
            os_sharpe_ratio_col = squared_sharpe_ratio(os_sample_col, col_factor, is_weights_col)
            os_sharpe_ratios_diff[i].append(os_sharpe_ratio_col - os_sharpe_ratio_row)

    # Calculate statistics
    stats = {'Model': [], 'FS Avg Diff': [], 'FS Med Diff': [], 'FS % < 0': [],
             'IS Avg Diff': [], 'IS Med Diff': [], 'IS % < 0': [],
             'OS Avg Diff': [], 'OS Med Diff': [], 'OS % < 0': []}

    for i, col_factor in enumerate(col_3_factors):
        stats['Model'].append(', '.join(col_factor))

        # Full Sample
        fs_diff = np.array(fs_sharpe_ratios_diff[i])
        stats['FS Avg Diff'].append(np.mean(fs_diff))
        stats['FS Med Diff'].append(np.median(fs_diff))
        stats['FS % < 0'].append(np.mean(fs_diff < 0) * 100)

        # In-Sample
        is_diff = np.array(is_sharpe_ratios_diff[i])
        stats['IS Avg Diff'].append(np.mean(is_diff))
        stats['IS Med Diff'].append(np.median(is_diff))
        stats['IS % < 0'].append(np.mean(is_diff < 0) * 100)

        # Out-of-Sample
        os_diff = np.array(os_sharpe_ratios_diff[i])
        stats['OS Avg Diff'].append(np.mean(os_diff))
        stats['OS Med Diff'].append(np.median(os_diff))
        stats['OS % < 0'].append(np.mean(os_diff < 0) * 100)

    # Convert stats to DataFrame
    stats_df = pd.DataFrame(stats)

    return stats_df


def squared_sharpe_ratio(sample, factor, weights=None):
    if weights is None:
        mu_f = sample[factor].mean().values.reshape(-1, 1) 
        sigma_f = sample[factor].cov().values
        sigma_f_inv = np.linalg.inv(sigma_f)

        # maximum squared Sharpe ratio
        sh2_f = mu_f.T @ sigma_f_inv @ mu_f
        sh2_f = sh2_f[0,0]

    else:
        weights = weights.flatten()
        returns = (sample[factor] * weights).sum(axis=1)
        mu_f = returns.mean()
        sigma_f = returns.std()

        # Calculate squared Sharpe ratio
        sh2_f = (mu_f / sigma_f) ** 2

    return sh2_f

def tangency_portfolio_weights(is_sample, factor):
    """
    Calculate the tangency portfolio weights.
    
    :param expected_returns: A vector of expected returns (mu)
    :param covariance_matrix: The covariance matrix (Sigma)
    :param risk_free_rate: The risk-free rate (rf)
    :return: A vector of tangency portfolio weights (t)

    source: https://bookdown.org/compfinezbook/introcompfinr/Efficient-portfolios-of.html - eq. 12.26
    """
    mu_f = is_sample[factor].mean().values.reshape(-1, 1) 
    sigma_f = is_sample[factor].cov().values
    sigma_f_inv = np.linalg.inv(sigma_f)

    # Inverse of the covariance matrix
    sigma_f_inv = np.linalg.inv(sigma_f)
    
    # Compute the numerator: Σ^(-1) * (μ - rf * 1) - mu_f is the derived from Mkt (in excess of risk free rate)
    numerator = sigma_f_inv @ (mu_f)
    
    # Compute the denominator: 1' * Σ^(-1) * (μ - rf * 1)
    denominator = np.ones(len(mu_f)) @ numerator
    
    # Tangency portfolio weights
    tangency_weights = numerator / denominator
    
    return tangency_weights

In [63]:
# Example usage
# row_factor = ['Mkt', 'S-F', 'H-F', 'R-F', 'C-F', 'U-F']
summary_df = pd.DataFrame()

col_3_factors = [model.split(', ') for model in top_3_actual['Model'].values]
row_factors = [model.split(', ') for model in table_3_summary['Model'].values]
# print(row_factors)
for row_factor in row_factors:
    stats_df = simulate_sharpe_ratios(master, row_factor, col_3_factors, n_simulations=100000)
    stats_df = stats_df.set_index('Model').T

    stats_df['Row_Model'] = ', '.join(row_factor)
    
    # Reshape the DataFrame to long format to stack the statistics
    stats_df = stats_df.reset_index().melt(id_vars=['index', 'Row_Model'], var_name='Column_Model', value_name='Value')
    stats_df = stats_df.pivot_table(index=['Row_Model', 'Column_Model'], columns='index', values='Value')
    stats_df.columns = [f'{col}' for col in stats_df.columns]

    # Concatenate with the summary DataFrame
    summary_df = pd.concat([summary_df, stats_df])

# Reset the index to get a clean final table
summary_df = summary_df.reset_index()

# Display the final summary table
summary_df.to_excel('./result/table4.xlsx', index=False)
summary_df

,Row_Model,Column_Model,FS % < 0,FS Avg Diff,FS Med Diff,IS % < 0,IS Avg Diff,IS Med Diff,OS % < 0,OS Avg Diff,OS Med Diff
0,"Mkt, SMB, HML, RMW, CMA, UMD","Mkt, S-F, H_s-F, R_s-F, C_s-F, U_s-F",44.929,0.006327,0.006318,44.173,0.005345,0.005305,42.167,0.006705,0.006431
1,"Mkt, SMB, HML, RMW, CMA, UMD","Mkt, SMB, HML_s, RMW_s, CMA_s, UMD_s",14.842,0.058698,0.058096,1.643,0.059902,0.057360,1.583,0.056999,0.054392
2,"Mkt, SMB, HML_s, RMW_s, CMA_s, UMD_s","Mkt, S-F, H_s-F, R_s-F, C_s-F, U_s-F",82.230,-0.052529,-0.051794,94.060,-0.054603,-0.051976,93.739,-0.050044,-0.047837
3,"Mkt, SMB, HML_s, RMW_s, CMA_s, UMD_s","Mkt, SMB, HML, RMW, CMA, UMD",85.345,-0.058904,-0.057866,98.343,-0.059814,-0.057273,98.404,-0.056851,-0.054164
4,"Mkt, S-F, H-F, R-F, C-F, U-F","Mkt, S-F, H_s-F, R_s-F, C_s-F, U_s-F",39.968,0.012571,0.012276,32.714,0.012870,0.012173,31.960,0.012528,0.011783
5,"Mkt, S-F, H-F, R-F, C-F, U-F","Mkt, SMB, HML, RMW, CMA, UMD",45.146,0.006247,0.006037,38.509,0.007471,0.007164,40.945,0.005438,0.005022
6,"Mkt, S-F, H-F, R-F, C-F, U-F","Mkt, SMB, HML_s, RMW_s, CMA_s, UMD_s",11.925,0.065236,0.063928,3.116,0.067500,0.064327,3.188,0.062473,0.059789
7,"Mkt, S-F, H_s-F, R_s-F, C_s-F, U_s-F","Mkt, SMB, HML, RMW, CMA, UMD",54.932,-0.006119,-0.006141,55.789,-0.005288,-0.005138,57.816,-0.006987,-0.006440
8,"Mkt, S-F, H_s-F, R_s-F, C_s-F, U_s-F","Mkt, SMB, HML_s, RMW_s, CMA_s, UMD_s",17.809,0.052558,0.051844,5.841,0.054437,0.051979,6.232,0.049835,0.047683
9,"Mkt, S-F, L-F, W-F, A-F, D-F","Mkt, S-F, H_s-F, R_s-F, C_s-F, U_s-F",7.049,0.064401,0.063337,7.763,0.065702,0.063014,6.231,0.063125,0.060337


### Table 5

In [18]:
def quote_factors(factors):
    """Quote factor names that include special characters like '-'."""
    return [f"Q('{factor}')" if '-' in factor else factor for factor in factors]

def spanning_regression(data, factors):
    """ Perform spanning regression for each factor in the model."""
    
    results = []
    sh2_f = top_3_actual[top_3_actual['Model'] == ', '.join(factors)]['Actual'].iloc[0]  # Calculate Sh^2(f) for the model

    for i, factor in enumerate(factors):
        # Define the LHS factor and the RHS factors (all others)
        rhs_factors = [f for j, f in enumerate(factors) if j != i]
        quoted_rhs_factors = quote_factors(rhs_factors)
        quoted_lhs_factor = f"Q('{factor}')" if '-' in factor else factor
        formula = f"{quoted_lhs_factor} ~ {' + '.join(quoted_rhs_factors)}"
        print(f"Running regression: {formula}")
        
        model = smf.ols(formula, data=data).fit()

        # Extract regression results
        intercept = model.params['Intercept']  # Ensure 'Intercept' is correctly referenced
        t_stat = model.tvalues['Intercept']    # Ensure 'Intercept' is correctly referenced
        r_squared = model.rsquared
        std_error = model.resid.std()
        
        # Calculate the marginal contribution a^2 / s^2(e)
        marginal_contribution = (intercept ** 2) / (std_error ** 2)
        
        # Collect results for this factor
        result = {
            'LHS': factor,
            'a': intercept,
        }

        # Add slopes for the other factors
        for f in rhs_factors:
            param_name = f"Q('{f}')" if '-' in f else f
            result[f] = model.params[param_name]
        
        stat = {
            't(a)': t_stat,
            'R2': r_squared,
            's(e)': std_error,
            'Sh2(f)': sh2_f,
            'a2/s2(e)': marginal_contribution
        }

        result.update(stat)
        
        results.append(result)
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

In [19]:
factor_a = ['Mkt', 'SMB', 'HML', 'RMW', 'CMA', 'UMD']
factor_b = ['Mkt', 'SMB', 'HML_s', 'RMW_s', 'CMA_s', 'UMD_s']
factor_c = ['Mkt', 'S-F', 'H_s-F', 'R_s-F', 'C_s-F', 'U_s-F']


t5_panel_a_comb = spanning_regression(master[factor_a], factor_a)
t5_panel_b_small = spanning_regression(master[factor_b], factor_b)
t5_panel_c_small_long = spanning_regression(master[factor_c], factor_c)

Running regression: Mkt ~ SMB + HML + RMW + CMA + UMD
Running regression: SMB ~ Mkt + HML + RMW + CMA + UMD
Running regression: HML ~ Mkt + SMB + RMW + CMA + UMD
Running regression: RMW ~ Mkt + SMB + HML + CMA + UMD
Running regression: CMA ~ Mkt + SMB + HML + RMW + UMD
Running regression: UMD ~ Mkt + SMB + HML + RMW + CMA
Running regression: Mkt ~ SMB + HML_s + RMW_s + CMA_s + UMD_s
Running regression: SMB ~ Mkt + HML_s + RMW_s + CMA_s + UMD_s
Running regression: HML_s ~ Mkt + SMB + RMW_s + CMA_s + UMD_s
Running regression: RMW_s ~ Mkt + SMB + HML_s + CMA_s + UMD_s
Running regression: CMA_s ~ Mkt + SMB + HML_s + RMW_s + UMD_s
Running regression: UMD_s ~ Mkt + SMB + HML_s + RMW_s + CMA_s
Running regression: Mkt ~ Q('S-F') + Q('H_s-F') + Q('R_s-F') + Q('C_s-F') + Q('U_s-F')
Running regression: Q('S-F') ~ Mkt + Q('H_s-F') + Q('R_s-F') + Q('C_s-F') + Q('U_s-F')
Running regression: Q('H_s-F') ~ Mkt + Q('S-F') + Q('R_s-F') + Q('C_s-F') + Q('U_s-F')
Running regression: Q('R_s-F') ~ Mkt + Q('S

In [20]:
t5_panel_a_comb

,LHS,a,SMB,HML,RMW,CMA,UMD,t(a),R2,s(e),Sh2(f),a2/s2(e),Mkt
0,Mkt,0.878919,0.242291,0.026394,-0.351644,-0.880191,-0.119180,5.507711,0.250846,3.841186,0.141268,0.052356,NaN
1,SMB,0.303558,NaN,0.076423,-0.438169,-0.158668,0.032114,2.590518,0.167615,2.769689,0.141268,0.012012,0.125971
2,HML,0.061563,0.037023,NaN,0.171834,0.988082,-0.117205,0.751162,0.524274,1.927774,0.141268,0.001020,0.006648
3,RMW,0.372317,-0.227743,0.184358,NaN,-0.345105,0.057570,4.452291,0.193053,1.996792,0.141268,0.034766,-0.095025
4,CMA,0.224460,-0.036271,0.466245,-0.151782,NaN,0.038085,4.036390,0.556989,1.324240,0.141268,0.028731,-0.104612
5,UMD,0.745395,0.068484,-0.515934,0.236206,0.355290,NaN,4.398980,0.088168,4.044641,0.141268,0.033964,-0.132139


In [21]:
t5_panel_b_small

,LHS,a,SMB,HML_s,RMW_s,CMA_s,UMD_s,t(a),R2,s(e),Sh2(f),a2/s2(e),Mkt
0,Mkt,0.920261,0.270287,-0.300325,-0.089246,-0.452315,-0.138128,5.536259,0.225978,3.904420,0.198995,0.055553,NaN
1,SMB,0.323654,NaN,-0.158610,-0.264882,0.127800,-0.034177,2.706944,0.174378,2.758416,0.198995,0.013767,0.134906
2,HML_s,0.040603,-0.082144,NaN,0.524729,1.049993,-0.095415,0.469238,0.609119,1.985102,0.198995,0.000418,-0.077633
3,RMW_s,0.404365,-0.141691,0.541974,NaN,-0.826563,0.036567,4.676561,0.412289,2.017456,0.198995,0.040173,-0.023828
4,CMA_s,0.277770,0.028354,0.449798,-0.342819,NaN,0.041583,5.000165,0.565602,1.299268,0.198995,0.045706,-0.050087
5,UMD_s,0.991784,-0.075322,-0.406033,0.150658,0.413078,NaN,5.696588,0.057246,4.095014,0.198995,0.058657,-0.151942


In [22]:
t5_panel_c_small_long

,LHS,a,S-F,H_s-F,R_s-F,C_s-F,U_s-F,t(a),R2,s(e),Sh2(f),a2/s2(e),Mkt
0,Mkt,-0.245072,-0.397143,-0.058189,0.541870,0.329205,0.254724,-2.926512,0.801770,1.975900,0.14857,0.015384,NaN
1,S-F,-0.270431,NaN,0.006827,0.228660,0.649821,0.196872,-6.798263,0.975375,0.965871,0.14857,0.078392,-0.094898
2,H_s-F,0.182127,0.011472,NaN,0.406375,0.694696,-0.181388,3.441034,0.949742,1.252041,0.14857,0.021160,-0.023364
3,R_s-F,0.036680,0.519081,0.549002,NaN,-0.472197,0.204832,0.590875,0.939991,1.455265,0.14857,0.000635,0.293934
4,C_s-F,0.069668,0.637005,0.405271,-0.203905,NaN,0.124034,1.711341,0.975189,0.956300,0.14857,0.005307,0.077113
5,U_s-F,0.424447,0.547448,-0.300171,0.250906,0.351843,NaN,6.372004,0.931987,1.610639,0.14857,0.069446,0.169253


In [23]:
t5_panel_a_comb.to_excel('./result/table5_a.xlsx', index=False)
t5_panel_b_small.to_excel('./result/table5_b.xlsx', index=False)
t5_panel_c_small_long.to_excel('./result/table5_c.xlsx', index=False)

### Table 6

In [7]:
def tangency_portfolio_weights(is_sample, factor, model_type='spread'):
    """
    Calculate the tangency portfolio weights.
    
    :param expected_returns: A vector of expected returns (mu)
    :param covariance_matrix: The covariance matrix (Sigma)
    :param risk_free_rate: The risk-free rate (rf)
    :return: A vector of tangency portfolio weights (t)

    source: https://bookdown.org/compfinezbook/introcompfinr/Efficient-portfolios-of.html - eq. 12.26
    """
    mu_f = is_sample[factor].mean().values.reshape(-1, 1) 
    sigma_f = is_sample[factor].cov().values
    sigma_f_inv = np.linalg.inv(sigma_f)

    # Inverse of the covariance matrix
    sigma_f_inv = np.linalg.inv(sigma_f)
    
    # Compute the numerator: Σ^(-1) * (μ - rf * 1) - mu_f is the derived from Mkt (in excess of risk free rate)
    numerator = sigma_f_inv @ (mu_f)
    
    # Compute the denominator: 1' * Σ^(-1) * (μ - rf * 1)
    denominator = np.ones(len(mu_f)) @ numerator
    
    # Tangency portfolio weights (initial weights before scaling)
    tangency_weights = numerator / denominator * 100
    
    if model_type == 'spread':
        # For spread factor models, scale all weights so that the market (Mkt) is 100%
        mkt_weight = tangency_weights[0, 0]
        scaled_weights = (tangency_weights / mkt_weight) * 100
        leverage = np.sum(np.abs(scaled_weights[1:])) / 100  # Exclude market weight from leverage calculation
    else:
        # For excess return models, scale so that the sum of weights is 100%
        # total_weight_sum = np.sum(np.abs(tangency_weights))
        scaled_weights = tangency_weights
        leverage = np.sum(np.abs(scaled_weights[scaled_weights < 0])) / 100  # Leverage based on short positions
    
    # Calculate Sh^2(f)
    sh2_f = (mu_f.T @ sigma_f_inv @ mu_f)[0, 0]
    
    return sh2_f, scaled_weights.flatten(), leverage

In [8]:
all_factors = [model.split(', ') for model in table_3_summary['Model'].values]
all_factors

[['Mkt', 'SMB', 'HML', 'RMW', 'CMA', 'UMD'],
 ['Mkt', 'SMB', 'HML_s', 'RMW_s', 'CMA_s', 'UMD_s'],
 ['Mkt', 'S-F', 'H-F', 'R-F', 'C-F', 'U-F'],
 ['Mkt', 'S-F', 'H_s-F', 'R_s-F', 'C_s-F', 'U_s-F'],
 ['Mkt', 'S-F', 'L-F', 'W-F', 'A-F', 'D-F'],
 ['Mkt', 'S-F', 'L_s-F', 'W_s-F', 'A_s-F', 'D_s-F']]

In [9]:
# Initialize an empty DataFrame to store the results
result_df = pd.DataFrame()

# Loop over each factor set (model)
for factors in all_factors:
    # Select the relevant data for the current set of factors
    data = master[factors]  # Assuming `master` is the DataFrame containing your data

    # Determine model type (spread or excess)
    model_type = 'spread' if 'SMB' in factors else 'excess'
    # Calculate weights and leverage for the current model
    sh2_f, weights, leverage = tangency_portfolio_weights(data, factors, model_type=model_type)

    mkt_weight = 100 if model_type == 'spread' else weights[0]
    # Prepare the result for the current model
    result = {
        'Model': ', '.join(factors),
        'Sh2(f)': sh2_f,
        'Mkt': mkt_weight,  # By definition, Mkt is scaled to 100%
        'Size': weights[1],
        'Value': weights[2],
        'Prof': weights[3],
        'Inv': weights[4],
        'Mom': weights[5],
        'Leverage': leverage
    }
    
    # Append the result to the DataFrame
    result_df = pd.concat([result_df, pd.DataFrame([result])], ignore_index=True)

# Display the final results
result_df

,Model,Sh2(f),Mkt,Size,Value,Prof,Inv,Mom,Leverage
0,"Mkt, SMB, HML, RMW, CMA, UMD",0.141268,100.000000,66.429462,27.809244,156.757942,214.876117,76.490691,5.423635
1,"Mkt, SMB, HML_s, RMW_s, CMA_s, UMD_s",0.198995,100.000000,70.463498,17.068310,164.576499,272.577756,97.973450,6.226595
2,"Mkt, S-F, H-F, R-F, C-F, U-F",0.135936,-1221.656769,-707.725226,132.465151,517.959567,811.293268,567.664008,19.293820
3,"Mkt, S-F, H_s-F, R_s-F, C_s-F, U_s-F",0.148570,-304.010926,-1403.919244,562.682150,83.882085,368.953264,792.412672,17.079302
4,"Mkt, S-F, L-F, W-F, A-F, D-F",0.085411,372.055721,247.244349,-69.948128,-179.024899,-201.384945,-68.942099,5.193001
5,"Mkt, S-F, L_s-F, W_s-F, A_s-F, D_s-F",0.103723,124.060921,432.230774,-42.800109,-153.245262,-173.932170,-86.314154,4.562917


In [11]:
result_df.to_excel('./result/table6.xlsx', index=False)